# Getting Mexico's schools and their careers

The objective of this repository is to get the data of Mexico's schools and their careers from an API and then saving it to a database using Entity Framework Core.

## Calling the API

First we are going to call the API. The data source is [DataMéxico](https://datamexico.org/en). I got the API link after playing a bit with their [Tesseract UI](https://api.datamexico.org/ui/), I'm only concerned about the careers that every institution offers, as well as the academic degree of every career.

This will get the data in Spanish, if you want to get it in English, change the `locale` parameter to `en`.

In [ ]:
using System.Net.Http;

var client = new HttpClient();
var locale = "es";
var response = await client.GetAsync(
    $"https://api.datamexico.org/tesseract/data.jsonrecords?cube=anuies_enrollment&drilldowns=Academic+Degree%2CCareer%2CInstitution&locale={locale}&measures=Students&parents=false&sparse=true");
var rawContent = await response.Content.ReadAsStringAsync();

If we open and inspect the API GET URL, we'll see that the response is a list of JSONs:

```json
{
    "data" : [
        {
            "Academic Degree ID": 8,
            "Academic Degree": "Estudios Técnicos o Comerciales con Preparatoria Terminada",
            "Career ID": 11200001,
            "Career": "Tecnico superior universitario en gestion de educacion y capacitacion",
            "Institution ID": 3345,
            "Institution": "Instituto Consorcio Clavijero",
            "Students": 13
        },
        ...
    ]
}
```

## Parsing the API response

Now we know how to access the list and the names of the properties:

In [ ]:
#r "nuget:Newtonsoft.Json"

using Newtonsoft.Json.Linq;

var objects = JObject.Parse(rawContent)["data"];

Installed Packages Newtonsoft.Json, 13.0.1

In [ ]:
objects.First().ToString()

{
  "Academic Degree ID": 8,
  "Academic Degree": "Estudios Técnicos o Comerciales con Preparatoria Terminada",
  "Career ID": 11000059,
  "Career": "Licencia profesional en ciencias de la educacion",
  "Institution ID": 2965,
  "Institution": "Universidad Unilider S.C.",
  "Students": 6
}

In [ ]:
objects.First()["Career"].ToString()

Licencia profesional en ciencias de la educacion

## Referencing models and database context

Before processing the data to organize it as needed, we first need to import the class library project *Database*.
I used Entity Framework Core code first approach to map the objects to the database, so the DLL output of the project already contains the models.

Inside the project folder, add a `.connection` file and put your connection string there.
See [`example.connection`](https://github.com/netotz/mexico-institutions-careers/blob/main/Database/example.connection).

The `SchoolsContext` class configures different table and property names for the models, according to my specific need.
Personally I would prefer to stick with the same class and property names, so the `OnModelCreating` method would be simpler.

Follow [these instructions](https://docs.microsoft.com/en-us/ef/core/managing-schemas/migrations/?tabs=dotnet-core-cli) to migrate the code to your database.
By doing this the project builds so the DLL will be already available.

In [ ]:
#r ".\Database\bin\Debug\net5.0\Database.dll"
#r "nuget:Microsoft.EntityFrameworkCore"
#r "nuget:Microsoft.EntityFrameworkCore.SqlServer"

using Database;

Installed Packages microsoft.data.sqlclient, 2.0.1 microsoft.data.sqlclient.sni.runtime, 2.0.1 Microsoft.EntityFrameworkCore, 5.0.8 microsoft.entityframeworkcore.abstractions, 5.0.8 microsoft.entityframeworkcore.relational, 5.0.8 Microsoft.EntityFrameworkCore.SqlServer, 5.0.8 microsoft.extensions.caching.abstractions, 5.0.0 microsoft.extensions.caching.memory, 5.0.0 microsoft.extensions.configuration.abstractions, 5.0.0 microsoft.extensions.dependencyinjection, 5.0.2 microsoft.extensions.dependencyinjection.abstractions, 5.0.0 microsoft.extensions.logging, 5.0.0 microsoft.extensions.logging.abstractions, 5.0.0 microsoft.extensions.options, 5.0.0 microsoft.extensions.primitives, 5.0.0 microsoft.identity.client, 4.14.0 microsoft.identitymodel.jsonwebtokens, 5.6.0 microsoft.identitymodel.logging, 5.6.0 microsoft.identitymodel.protocols, 5.6.0 microsoft.identitymodel.protocols.openidconnect, 5.6.0 microsoft.identitymodel.tokens, 5.6.0 microsoft.win32.systemevents, 4.7.0 system.configuration.configurationmanager, 4.7.0 system.diagnostics.diagnosticsource, 5.0.1 system.drawing.common, 4.7.0 system.identitymodel.tokens.jwt, 5.6.0 system.runtime.caching, 4.7.0 system.security.cryptography.protecteddata, 4.7.0 system.security.permissions, 4.7.0 system.windows.extensions, 4.7.0

## Processing and organizing the data

Let's start by filtering unique records of institutions, degrees, and careers:

In [ ]:
var institutions = objects
    .GroupBy(o => o["Institution ID"])
    .Select(g => g.First())
    .Select(o => new Institution {
        Id = (int)o["Institution ID"],
        Name = o["Institution"].ToString()
    })
    .OrderBy(i => i.Name)
    .ToList();

institutions

index,Id,Name,Careers,InstitutionCareers
0,510,"17, Instituto De Estudios Críticos",<null>,<null>
1,3577,1972 Año De Juárez,<null>,<null>
2,178,Academia De Arte Culinaria Internacional,<null>,<null>
3,509,Academia De Arte De Florencia,<null>,<null>
4,895,Academia De Arte Y Diseño,<null>,<null>
5,3426,Academia De Ciencias Sociales Y Desarrollo Humano Acanits,<null>,<null>
6,2724,Academia De Estudios Superiores Constantino,<null>,<null>
7,2341,Academia De Integración Jurídica De Oaxaca,<null>,<null>
8,3598,Academia De Música Fermatta,<null>,<null>
9,1438,Academia De Música Solfeggio,<null>,<null>


In [ ]:
var careers = objects
    .GroupBy(o => o["Career ID"])
    .Select(g => g.First())
    .Select(o => new Career {
        Id = (int)o["Career ID"],
        Name = o["Career"].ToString(),
        DegreeId = (int)o["Academic Degree ID"]
    })
    .OrderBy(c => c.Name)
    .ToList();

careers

index,Id,Name,DegreeId,Degree,Institutions,InstitutionCareers
0,33100537,Abogado,11,<null>,<null>,<null>
1,33100536,"Abogado, notario y actuario",11,<null>,<null>,<null>
2,53200029,Actuario,11,<null>,<null>,<null>
3,91301011,Alta especialidad en anestesia de alto riesgo y medicina perioperatoria,12,<null>,<null>,<null>
4,91332008,Alta especialidad en broncoscopia y neumologia intervencionista,12,<null>,<null>,<null>
5,91335008,Alta especialidad en glaucoma,12,<null>,<null>,<null>
6,91344007,"Alta especialidad en minima invasion, endourologia y cirugia robotica en urologia",12,<null>,<null>,<null>
7,91309005,Alta especialidad en oculoplastica,12,<null>,<null>,<null>
8,91335007,Alta especialidad en retina y vitrio,12,<null>,<null>,<null>
9,73100186,Arquitecto,11,<null>,<null>,<null>


In [ ]:
var degrees = objects
    .GroupBy(o => o["Academic Degree ID"])
    .Select(g => g.First())
    .Select(o => new Degree {
        Id = (int)o["Academic Degree ID"],
        Name = o["Academic Degree"].ToString(),
    })
    .OrderBy(d => d.Name)
    .ToList();

degrees

index,Id,Name,Careers
0,14,Doctorado,<null>
1,12,Especialidad,<null>
2,8,Estudios Técnicos o Comerciales con Preparatoria Terminada,<null>
3,11,Licenciatura,<null>
4,13,Maestría,<null>
5,10,Normal de Licenciatura,<null>


Now that we have all the data of each table, let's create a query to get the many-to-many relationship between institutions and the careers each one offers:

In [ ]:
var relations = objects
    .Select(o => new InstitutionCareer {
        InstitutionId = (int)o["Institution ID"],
        CareerId = (int)o["Career ID"]
    })
    .GroupBy(r => new { r.InstitutionId, r.CareerId })
    .Select(g => g.First())
    .ToList();

relations

index,InstitutionId,Institution,CareerId,Career
0,2965,<null>,11000059,<null>
1,3345,<null>,11200001,<null>
2,1187,<null>,12000001,<null>
3,1432,<null>,12602001,<null>
4,2192,<null>,12602001,<null>
5,597,<null>,12602002,<null>
6,2229,<null>,12603001,<null>
7,2997,<null>,12603002,<null>
8,2724,<null>,12603003,<null>
9,2659,<null>,12603004,<null>


## Uploading to database

It's time to upload the data we just filtered and organized to the database. First, we need to create an instance of our database context, `SchoolsContext`, and with it we can use all the power provided by Entity Framework Core, including adding the previous records and saving the changes:

In [ ]:
using (var context = new SchoolsContext()) {
    await context.Institutions.AddRangeAsync(institutions);
    await context.Degrees.AddRangeAsync(degrees);
    await context.Careers.AddRangeAsync(careers);
    await context.InstitutionCareers.AddRangeAsync(relations);
    
    await context.SaveChangesAsync();
}

## Retrieving from database

Finally let's check if our data was successfully uplodaded.
There were no errors so if you check the tables in your own database server you should be able to verify it, but for demonstration purposes I'm going to use the context to get some data:

In [ ]:
using Microsoft.EntityFrameworkCore;
using System.Linq;

public class InstitutionDto {
    public class CareerDto {
        public string Name { get; set; }
        public string Degree { get; set; }
    }

    public string Name { get; set; }
    public List<CareerDto> Careers { get; set; }
}

InstitutionDto uanl;
using (var context = new SchoolsContext()) {
    uanl = await context.Institutions
        .Where(i => i.Name == "Universidad Autónoma De Nuevo León")
        .Select(i => new InstitutionDto {
            Name = i.Name,
            Careers = i.Careers
                .Select(c => new InstitutionDto.CareerDto {
                    Name = c.Name,
                    Degree = c.Degree.Name
                })
                .ToList()
        })
        .SingleOrDefaultAsync();
}

Now we can do any query we want, as getting a list of all the bachelors offered by an university, in this case the UANL:

In [ ]:
uanl.Name

Universidad Autónoma De Nuevo León

In [ ]:
uanl.Careers
    .Where(c => c.Degree == "Licenciatura")
    .OrderBy(c => c.Name)

index,Name,Degree
0,Arquitecto,Licenciatura
1,Biologo,Licenciatura
2,Cirujano dentista,Licenciatura
3,Ingenieria administrador de sistemas,Licenciatura
4,Ingenieria agronomo,Licenciatura
5,Ingenieria ambiental,Licenciatura
6,Ingenieria civil,Licenciatura
7,Ingenieria en aeronautica,Licenciatura
8,Ingenieria en agronegocios,Licenciatura
9,Ingenieria en biotecnologia,Licenciatura
